Play around with the BayesianGaussianMixture models

In [47]:
from sklearn.mixture import BayesianGaussianMixture
import pandas as pd 

In [48]:
data = pd.read_csv("../datasets/Adult/Real/real_adult_data.csv")
data = data.drop(columns=["Unnamed: 0"])
data = data.sample(n=5000)


In [49]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
21005,31,Private,188096,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,72,United-States,>50K
4485,63,Private,151210,7th-8th,4,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K
1081,30,Private,285295,Bachelors,13,Married-civ-spouse,Other-service,Wife,Asian-Pac-Islander,Female,0,0,40,Philippines,<=50K
24501,35,Private,28160,Bachelors,13,Married-spouse-absent,Exec-managerial,Unmarried,White,Female,0,0,40,United-States,<=50K
2371,58,Private,354024,HS-grad,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States,>50K


In [50]:
varname = "hours-per-week"
data[varname].describe()

count    5000.000000
mean       40.943600
std        11.708007
min         2.000000
25%        40.000000
50%        40.000000
75%        45.000000
max        99.000000
Name: hours-per-week, dtype: float64

In [51]:

def fit_bgmm(x):
    gm = BayesianGaussianMixture(
        n_components=3,
        covariance_type='full',
        weight_concentration_prior_type='dirichlet_process',  # special for B-GMM
        weight_concentration_prior=1e-3, # special
        n_init=5 , #10
        max_iter=2000, #1000
        warm_start=True,
        init_params="random",
    )
    gm.fit(x.reshape(-1, 1))

    valid_component_indicator = gm.weights_
    num_components = valid_component_indicator.sum()

    return num_components

def sequential_function(data, variables):
    output = []
    for v in variables:
        x = data[v].values
        output.append(fit_bgmm(x))
    return output 
    


In [52]:
varnames = ["fnlwgt", "hours-per-week", "education-num", "age", "capital-gain", "capital-loss"] 
# what is fnlwg? do we need that? -- seems very slow

In [53]:
%timeit sequential_function(data, varnames)

15.9 s ± 619 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
